In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pyportfolioopt

     |████████████████████████████████| 61 kB 6.7 MB/s 
     |████████████████████████████████| 2.8 MB 42.0 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions

In [ ]:
def get_optimal_weights(exp_returns: pd.Series, exp_cov:pd.DataFrame, vol_limit: float = None):
  ef = EfficientFrontier(exp_returns, exp_cov)
  if vol_limit is not None:
    weights = ef.efficient_risk(vol_limit)
  # If vol limit not specified, default to maximising return risk ratio
  else:
    weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
        )
  return pd.Series(weights)

## Baseline Optimisation

#### Uses rolling 30 days mean returns to calculate expected returns and rolling 252 days returns to calculate covariance

In [ ]:
ohlc_data = pd.read_csv("/content/drive/MyDrive/Portfolio Optimization [Fintech ML]/OHLC Data/ohlc_data.csv")
ohlc_data["Date"] = pd.to_datetime(ohlc_data["Date"])
ohlc_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2000-10-02,53.393299,55.143902,52.408588,55.143902,41.145496,4219239,AXP
1,2000-10-03,54.651543,54.870369,53.721539,53.830952,40.165859,3933737,AXP
2,2000-10-04,53.885654,54.104481,53.283890,53.283890,39.809448,2751628,AXP
3,2000-10-05,53.995068,54.706249,53.174477,53.393299,39.891190,3767623,AXP
4,2000-10-06,53.010357,53.174477,50.329750,50.712692,37.888451,6946990,AXP


In [ ]:
data = ohlc_data.pivot(index="Date",columns="ticker",values="Adj Close")

In [ ]:
ohlc_data

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2000-10-02,53.393299,55.143902,52.408588,55.143902,41.145496,4219239,AXP
1,2000-10-03,54.651543,54.870369,53.721539,53.830952,40.165859,3933737,AXP
2,2000-10-04,53.885654,54.104481,53.283890,53.283890,39.809448,2751628,AXP
3,2000-10-05,53.995068,54.706249,53.174477,53.393299,39.891190,3767623,AXP
4,2000-10-06,53.010357,53.174477,50.329750,50.712692,37.888451,6946990,AXP
...,...,...,...,...,...,...,...,...
137533,2020-12-23,172.960007,175.949997,172.300003,173.550003,173.550003,9994000,DIS
137534,2020-12-24,173.990005,174.580002,172.380005,173.729996,173.729996,2721000,DIS
137535,2020-12-28,175.750000,179.919998,175.050003,178.860001,178.860001,13145400,DIS
137536,2020-12-29,180.009995,180.199997,176.220001,177.300003,177.300003,6875400,DIS


In [ ]:
data = ohlc_data.pivot(index="Date",columns="ticker",values="Adj Close")
returns = data.pct_change()

In [ ]:
returns.head()

ticker,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-10-03,-0.079896,0.015209,-0.023809,0.024599,0.023723,0.013513,0.002163,-0.004747,-0.013002,0.023781,0.032986,-0.061538,0.004673,-0.007378,-0.015789,-0.012062,0.006237,0.025221,0.000000,-0.043341,0.037500,0.011019,-0.022472,-0.005899,0.005277,0.000000,-0.009459
2000-10-04,0.058822,-0.034644,-0.008873,0.020877,0.001783,0.041111,-0.023741,0.014309,-0.038968,0.032520,0.018487,0.034482,0.041861,-0.023649,-0.018684,0.016649,-0.008264,0.015958,-0.005913,-0.019889,-0.019579,0.019073,-0.015326,-0.026113,0.038058,0.016835,0.006821
2000-10-05,-0.066138,0.033948,0.002053,-0.001023,-0.003559,-0.017076,0.008106,0.034483,0.049686,-0.032620,-0.033003,-0.010382,-0.023810,0.030450,-0.024692,0.032750,0.004166,-0.023561,0.023789,-0.001128,0.056835,0.040999,0.003891,0.021938,0.057540,0.038079,-0.001355
2000-10-06,0.005666,-0.053471,-0.050205,-0.001024,-0.014286,-0.023887,-0.013158,-0.013636,-0.052231,-0.050000,-0.003413,0.024848,-0.025915,-0.008059,-0.035162,-0.011627,-0.008298,-0.011394,0.009959,0.003386,-0.077034,0.012842,-0.032300,-0.017889,-0.028916,-0.033492,-0.012212


In [ ]:
# Returns dataframe of expected returns as the mean of the past [window] days
def get_predicted_returns(window: int):
  return returns.rolling(window).mean().shift(1)["2016-01-01":]

# Returns multiindex dataframe where first level index is date, value is dataframe of covariance matrix calculated by past [window] number of days
def get_predicted_covariance(window:int):
  return returns.rolling(window).cov().unstack().shift(1).stack().query("Date > '2016-01-01'")

In [ ]:
exp_return= get_predicted_returns(30)
exp_cov = get_predicted_covariance(252)

In [ ]:
exp_return.head()

ticker,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.002452,0.001130,-0.000685,-0.000241,-0.000551,0.000508,-0.000209,-0.003000,-0.001601,0.001777,0.000531,0.000987,0.001895,0.000689,0.000056,0.001331,0.002385,-0.001058,-0.000178,0.001627,0.000877,0.001927,0.000292,0.001233,0.000885,0.000971,0.001094
2016-01-05,-0.003479,-0.000391,-0.002112,-0.001685,-0.000983,-0.000546,-0.001052,-0.004243,-0.002713,0.001385,-0.000368,0.000084,0.000919,-0.000422,-0.001610,0.000408,0.001749,-0.002200,-0.000716,0.000665,-0.000516,0.001122,-0.001135,0.000065,0.000411,-0.000501,0.000620
2016-01-06,-0.004737,-0.000131,-0.002897,-0.001766,-0.001189,-0.001004,-0.000268,-0.005077,-0.003179,0.001138,-0.000409,-0.000166,-0.000384,-0.000225,-0.001656,-0.000128,0.001981,-0.002264,-0.000321,0.000761,-0.000050,0.001088,-0.001083,0.002010,0.000589,-0.000687,0.001537
2016-01-07,-0.005536,-0.000524,-0.003676,-0.002331,-0.002247,-0.001603,-0.000917,-0.005637,-0.003723,-0.000088,-0.000973,-0.000762,-0.001473,-0.000636,-0.002078,0.000217,0.001577,-0.003055,-0.000668,0.000001,-0.002346,0.000941,-0.001749,0.000965,0.000558,-0.000795,0.002217
2016-01-08,-0.006509,-0.001853,-0.003893,-0.003492,-0.003339,-0.002203,-0.002473,-0.005742,-0.004351,-0.001624,-0.001626,-0.001322,-0.002549,-0.001118,-0.003101,-0.000750,0.000642,-0.003804,-0.000905,-0.001158,-0.003187,0.000583,-0.002486,0.000223,0.000668,-0.000183,0.002888


In [ ]:
exp_cov.head()

ticker                 AAPL      AMGN       AXP  ...        VZ       WBA       WMT
Date       ticker                                ...                              
2016-01-04 AAPL    0.000284  0.000116  0.000086  ...  0.000080  0.000107  0.000078
           AMGN    0.000116  0.000318  0.000106  ...  0.000075  0.000166  0.000091
           AXP     0.000086  0.000106  0.000182  ...  0.000065  0.000091  0.000053
           BA      0.000130  0.000130  0.000070  ...  0.000071  0.000110  0.000086
           CAT     0.000119  0.000107  0.000090  ...  0.000077  0.000080  0.000061

[5 rows x 27 columns]

In [ ]:
optimal_weights = exp_return.apply(lambda x: get_optimal_weights(x, exp_cov.loc[x.name].reset_index(level=0, drop=True)), axis=1)

In [ ]:
portfolio_returns = (optimal_weights * returns).dropna().sum(axis=1)

In [ ]:
portfolio_returns.head()

Date
2016-01-04   -0.024579
2016-01-05    0.004562
2016-01-06   -0.018165
2016-01-07   -0.023864
2016-01-08    0.003067
dtype: float64

In [ ]:
portfolio_returns.to_csv("baseline_return.csv")

## ML Optimization


#### Uses LSTM model to calculate expected returns and GARCH model to calculate covariance

In [ ]:
ml_exp_return = pd.read_csv("/content/drive/MyDrive/Portfolio Optimization [Fintech ML]/Forecast Data/forecast_return.csv").set_index("Date").sort_index(axis=1)
ml_exp_cov = pd.read_csv("/content/drive/MyDrive/Portfolio Optimization [Fintech ML]/Forecast Data/forecast_covariance.csv").set_index(["Date","ticker"])

In [ ]:
ml_exp_return.head()

,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.000119,0.000934,-0.000575,-0.193893,-0.000101,-0.176865,-0.001399,0.000852,0.000553,0.001078,0.000664,-0.000260,0.000408,-0.000586,0.000839,0.000143,-0.000075,0.000526,-0.000954,0.001507,0.001543,0.000191,0.000904,0.001337,0.000396,0.001208,-0.000218
2016-01-05,-0.000132,0.000969,-0.000577,-0.193893,-0.000098,-0.176865,-0.001371,0.000846,0.000581,0.001093,0.000663,-0.000260,0.000421,-0.000566,0.000837,0.000181,-0.000054,0.000526,-0.000945,0.001537,0.001506,0.000222,0.000931,0.001372,0.000410,0.001209,-0.000225
2016-01-06,-0.000164,0.000995,-0.000570,-0.193893,-0.000097,-0.176865,-0.001359,0.000857,0.000615,0.001115,0.000665,-0.000256,0.000426,-0.000546,0.000834,0.000218,-0.000033,0.000526,-0.000904,0.001555,0.001492,0.000249,0.000953,0.001416,0.000417,0.001210,-0.000230
2016-01-07,-0.000218,0.001027,-0.000551,-0.193893,-0.000099,-0.176865,-0.001301,0.000875,0.000657,0.001149,0.000667,-0.000245,0.000435,-0.000523,0.000831,0.000261,-0.000016,0.000526,-0.000873,0.001595,0.001466,0.000285,0.000980,0.001490,0.000428,0.001212,-0.000233
2016-01-08,-0.000321,0.001080,-0.000523,-0.193893,-0.000102,-0.176865,-0.001213,0.000896,0.000702,0.001206,0.000671,-0.000209,0.000458,-0.000490,0.000830,0.000322,0.000003,0.000525,-0.000841,0.001663,0.001440,0.000327,0.001024,0.001623,0.000442,0.001213,-0.000233


In [ ]:
ml_exp_cov.head()

AAPL      AMGN       AXP  ...        VZ       WBA       WMT
Date       ticker                                ...                              
2016-01-04 AAPL    0.000321  0.000153  0.000125  ...  0.000138  0.000162  0.000107
           AMGN    0.000153  0.000269  0.000151  ...  0.000102  0.000171  0.000069
           AXP     0.000125  0.000151  0.000160  ...  0.000094  0.000118  0.000059
           BA      0.000155  0.000156  0.000169  ...  0.000107  0.000132  0.000077
           CAT     0.000148  0.000154  0.000109  ...  0.000096  0.000130  0.000098

[5 rows x 27 columns]

In [ ]:
ml_optimal_weights = ml_exp_return.apply(lambda x: get_optimal_weights(x, ml_exp_cov.loc[x.name].reset_index(level=0, drop=True)), axis=1)

In [ ]:
ml_portfolio_returns = (ml_optimal_weights * returns).dropna().sum(axis=1)

In [ ]:
ml_portfolio_returns.head()

Date
2016-01-04   -0.017145
2016-01-05   -0.017219
2016-01-06   -0.024412
2016-01-07   -0.030735
2016-01-08   -0.004131
dtype: float64

In [ ]:
ml_portfolio_returns.to_csv("ML_return.csv")

## Volatility Limit Study

#### Observe how portfolios perform when we choose to set a volatility limit and maximise returns instead of maximising return risk ratio

In [90]:
# Set portfolio volatility limit to 10%
baseline_maxret_weights = exp_return.apply(lambda x: get_optimal_weights(x, exp_cov.loc[x.name].reset_index(level=0, drop=True), 0.1), axis=1)
ml_maxret_weights = ml_exp_return.apply(lambda x: get_optimal_weights(x, ml_exp_cov.loc[x.name].reset_index(level=0, drop=True), 0.1), axis=1)

In [93]:
baseline_maxret_returns = (baseline_maxret_weights * returns).dropna().sum(axis=1)
ml_maxret_returns = (ml_maxret_weights * returns).dropna().sum(axis=1)

In [94]:
baseline_maxret_returns.to_csv("baseline_maxret_return.csv")
ml_maxret_returns.to_csv("ML_maxret_return.csv")